In [1]:
from pysolar.solar import get_altitude, get_azimuth
import matplotlib.image as mpimg
from datetime import datetime, timedelta
from pytz import timezone
import numpy as np

def calculate_solar_positions(
    start_date: datetime,
    start_hour: int = 10,
    end_hour: int = 17,
    latitude: float = -0.2105367,
    longitude: float = -78.491614
):
    """Calcula posiciones solares y ángulos para una fecha específica y rango de horas.

    ## Parameters
    - start_date: Fecha y hora de inicio para la simulación (se usa solo la fecha, hora se define con start_hour).
    - start_hour: Hora de inicio del cálculo.
    - end_hour: Hora de fin del cálculo.
    - latitude: Latitud para la posición geográfica.
    - longitude: Longitud para la posición geográfica.

    ## Return
    - times: Lista de tiempos de simulación.
    - azimuths: Lista de ángulos azimutales.
    - elevations: Lista de ángulos de elevación.
    - beta: Lista de ángulos de rollo calculados.
    - phi: Lista de ángulos de inclinación calculados.
    """
    times = []
    azimuths = []
    elevations = []
    beta = []
    phi = []

    time_interval = timedelta(hours=0.33)

    # Crear el rango de tiempos basado en la hora de inicio y fin
    start_time = start_date.replace(hour=start_hour, minute=0, second=0, microsecond=0)
    end_time = start_date.replace(hour=end_hour, minute=0, second=0, microsecond=0)

    current_time = start_time
    while current_time <= end_time:
        az = get_azimuth(latitude, longitude, current_time)
        el = get_altitude(latitude, longitude, current_time)

        # Convertir a radianes para los cálculos
        az_rad = (az * np.pi) / 180
        el_rad = (el * np.pi) / 180

        val = np.cos(el_rad) * np.sin(az_rad)
        beta_rad = np.arcsin(val)
        beta_deg = (beta_rad * 180) / np.pi

        val_fi1 = -(np.cos(el_rad) * np.cos(az_rad)) / np.cos(beta_rad)
        x_rad = np.arcsin(val_fi1)
        x_deg = (x_rad * 180) / np.pi

        val_fi2 = np.sin(el_rad) / np.cos(beta_rad)
        y_rad = np.arccos(val_fi2)
        y_deg = (y_rad * 180) / np.pi

        times.append(current_time)
        azimuths.append(az)
        elevations.append(el)
        beta.append(beta_deg)
        phi.append(x_deg)  # Asumiendo que phi corresponde a x_deg; ajusta si es necesario

        current_time += time_interval

    return times, azimuths, elevations, beta, phi

In [2]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import tkinter as tk
import numpy as np

# Extraer los datos de tiempo, azimuth y elevación
#times = [pos[0] for pos in solar_positions]
#azimuths = [pos[1] for pos in solar_positions]
#elevations = [pos[2] for pos in solar_positions]
#beta = [sol[0] for sol in solucion]
#phi = [sol[1] for sol in solucion]

fecha = input("Ingrese una fecha en formato año-mes-día (ejemplo: 2024-08-11): ")
anio, mes, dia = fecha.split('-')
rango = input("Ingrese el rango de la simulación (ejemplo: 10-17): ")
inicio, fin = rango.split('-')

start_date = datetime(int(anio), int(mes), int(dia), tzinfo=timezone("America/Guayaquil"))
times, azimuths, elevations, beta, phi = calculate_solar_positions(start_date, start_hour=int(inicio), end_hour=int(fin))

In [14]:
# Supongamos que ya has calculado los valores de times, azimuths, elevations, beta, y phi
# Aquí está el código para imprimirlos en un formato legible.

print("Solar Position Data:\n")

print("Times: ", end="")
print(", ".join([f"{time.strftime('%H:%M')}" for time in times]))

print("\nAzimuths: ", end="")
print(", ".join([f"{azimuth:.2f}°" for azimuth in azimuths]))

print("\nElevations: ", end="")
print(", ".join([f"{elevation:.2f}°" for elevation in elevations]))

print("\nBeta (Ángulo de Inclinación): ", end="")
print(", ".join([f"{b:.2f}°" for b in beta]))

print("\nPhi (Ángulo de Orientación): ", end="")
print(", ".join([f"{p:.2f}°" for p in phi]))

Solar Position Data:

Times: 08:00, 08:19, 08:39, 08:59, 09:19, 09:39, 09:58, 10:18, 10:38, 10:58, 11:18, 11:37, 11:57, 12:17, 12:37, 12:57, 13:16, 13:36, 13:56, 14:16, 14:36, 14:55, 15:15, 15:35, 15:55, 16:15, 16:34, 16:54, 17:14, 17:34, 17:54

Azimuths: 114.18°, 115.32°, 116.75°, 118.54°, 120.76°, 123.54°, 127.04°, 131.46°, 137.09°, 144.23°, 153.17°, 163.98°, 176.22°, 188.83°, 200.60°, 210.68°, 218.88°, 225.38°, 230.49°, 234.51°, 237.70°, 240.24°, 242.28°, 243.92°, 245.23°, 246.27°, 247.09°, 247.70°, 248.14°, 248.43°, 248.56°

Elevations: 27.06°, 31.55°, 35.99°, 40.37°, 44.67°, 48.86°, 52.90°, 56.74°, 60.29°, 63.43°, 66.01°, 67.83°, 68.69°, 68.47°, 67.21°, 65.06°, 62.24°, 58.91°, 55.24°, 51.31°, 47.20°, 42.96°, 38.63°, 34.22°, 29.75°, 25.24°, 20.71°, 16.15°, 11.59°, 7.04°, 2.57°

Beta (Ángulo de Inclinación): 54.33°, 50.38°, 46.26°, 42.01°, 37.67°, 33.25°, 28.78°, 24.27°, 19.72°, 15.16°, 10.57°, 5.98°, 1.37°, -3.23°, -7.83°, -12.42°, -17.00°, -21.56°, -26.10°, -30.60°, -35.05°, -39.4

In [4]:
x_labels = []
for time in times:
        x_labels.append(f"{time.hour}:{time.minute}")

## Simulación con los puntos

In [5]:
# Convertir los tiempos a números (por simplicidad en la gráfica 3D)
time_numbers = np.arange(len(times))

# Configurar la ventana de Tkinter
root = tk.Tk()
root.title("Simulación de Ángulos Solares")

# Crear un marco para la gráfica 3D
frame_3d = tk.Frame(root)
frame_3d.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

# Crear la figura y el eje 3D
fig3 = plt.Figure(figsize=(13, 7), dpi=100)
ax3 = fig3.add_subplot(111, projection='3d')

# Configurar los ejes
ax3.set_xlabel('Time')
ax3.set_ylabel('Azimuth')
ax3.set_zlabel('Elevation')
ax3.set_title('Trayectoria del sol')

# Configurar límites fijos para evitar redimensionado
ax3.set_xlim(0, len(times)-1)
ax3.set_ylim(min(azimuths), max(azimuths))
ax3.set_zlim(min(elevations), max(elevations))

# Crear una función para generar la esfera que representa el sol
def create_sun(ax, center, radius=2, color='yellow'):
    # Crear una esfera 3D
    u = np.linspace(0, 2 * np.pi, 30)
    v = np.linspace(0, np.pi, 15)
    x = radius * np.outer(np.cos(u), np.sin(v)) + center[0]
    y = radius * np.outer(np.sin(u), np.sin(v)) + center[1]
    z = radius * np.outer(np.ones(np.size(u)), np.cos(v)) + center[2]

    # Añadir la esfera al eje 3D
    ax.plot_surface(x, y, z, color=color, shade=True)

# Función de inicialización para la animación 3D
def init_3d():
    return []

# Función de actualización para la animación 3D
def update_3d(frame):
    ax3.cla()  # Limpiar el eje antes de actualizar

    # Re-dibujar la trayectoria hasta el punto actual
    ax3.plot(time_numbers[:frame+1], azimuths[:frame+1], elevations[:frame+1], 'yo-', markersize=5)

    # Crear la esfera 3D para representar el sol en la posición actual
    sun_position = [time_numbers[frame], azimuths[frame], elevations[frame]]
    create_sun(ax3, sun_position, radius=0.5, color='yellow')

    # Reestablecer límites para evitar redimensionado
    ax3.set_xlim(0, len(times)-1)
    ax3.set_ylim(min(azimuths), max(azimuths))
    ax3.set_zlim(min(elevations), max(elevations))
    ax3.set_xlabel('Time')
    ax3.set_ylabel('Azimuth')
    ax3.set_zlabel('Elevation')
    ax3.set_title('Trayectoria del sol')

    return []

# Crear el lienzo para la figura 3D
canvas3 = FigureCanvasTkAgg(fig3, master=frame_3d)
canvas3.draw()
canvas3.get_tk_widget().pack(fill=tk.BOTH, expand=True)

interval_time_ms = 500
ani3 = animation.FuncAnimation(fig3, update_3d, frames=len(times), init_func=init_3d, interval=interval_time_ms, blit=True)

# Iniciar la interfaz gráfica de Tkinter
root.mainloop()

## Simulación Solo el SOL

In [6]:
# Configurar la ventana de Tkinter
root = tk.Tk()
root.title("Simulación de Ángulos Solares")

# Crear un marco para la gráfica 3D
frame_3d = tk.Frame(root)
frame_3d.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

# Crear la figura y el eje 3D
fig3 = plt.Figure(figsize=(13, 7), dpi=100)
ax3 = fig3.add_subplot(111, projection='3d')

# Configurar los ejes
ax3.set_xlabel('Time')
ax3.set_ylabel('Azimuth')
ax3.set_zlabel('Elevation')
ax3.set_title('Trayectoria del sol')

# Configurar límites fijos para evitar redimensionado
ax3.set_xlim(0, len(times)-1)
ax3.set_ylim(min(azimuths), max(azimuths))
ax3.set_zlim(min(elevations), max(elevations))

# Crear una función para generar la esfera que representa el sol
def create_sun(ax, center, radius=2.0, color='yellow'):  # Aquí se ajusta el tamaño de la esfera
    # Crear una esfera 3D
    u = np.linspace(0, 2 * np.pi, 50)  # Aumentar la resolución para una mejor apariencia
    v = np.linspace(0, np.pi, 50)
    x = radius * np.outer(np.cos(u), np.sin(v)) + center[0]
    y = radius * np.outer(np.sin(u), np.sin(v)) + center[1]
    z = radius * np.outer(np.ones(np.size(u)), np.cos(v)) + center[2]

    # Añadir la esfera al eje 3D con sombreado
    ax.plot_surface(x, y, z, color=color, shade=True, rstride=1, cstride=1, linewidth=0)

# Función de inicialización para la animación 3D
def init_3d():
    return []

# Función de actualización para la animación 3D
def update_3d(frame):
    ax3.cla()  # Limpiar el eje antes de actualizar

    # Re-dibujar la trayectoria hasta el punto actual, sin los puntos amarillos
    ax3.plot(time_numbers[:frame+1], azimuths[:frame+1], elevations[:frame+1], 'y-', markersize=0)

    # Crear la esfera 3D para representar el sol en la posición actual
    sun_position = [time_numbers[frame], azimuths[frame], elevations[frame]]
    create_sun(ax3, sun_position, radius=2.0, color='yellow')  # Ajusta el tamaño de la esfera aquí

    # Reestablecer límites para evitar redimensionado
    ax3.set_xlim(0, len(times)-1)
    ax3.set_ylim(min(azimuths), max(azimuths))
    ax3.set_zlim(min(elevations), max(elevations))
    ax3.set_xlabel('Time')
    ax3.set_ylabel('Azimuth')
    ax3.set_zlabel('Elevation')
    ax3.set_title('Trayectoria del sol')

    return []

# Crear el lienzo para la figura 3D
canvas3 = FigureCanvasTkAgg(fig3, master=frame_3d)
canvas3.draw()
canvas3.get_tk_widget().pack(fill=tk.BOTH, expand=True)

interval_time_ms = 500
ani3 = animation.FuncAnimation(fig3, update_3d, frames=len(times), init_func=init_3d, interval=interval_time_ms, blit=True)

# Iniciar la interfaz gráfica de Tkinter
root.mainloop()

## Simulación SOL angúlo Polar

In [15]:
# Convertir los azimuths de grados a radianes para el gráfico polar
azimuths_rad = np.radians(azimuths)

# Cargar la imagen del sol
sun_img = mpimg.imread('./Interfaz-SeguidorSolar/images/sol4.jpg')

# Configurar la ventana de Tkinter
root = tk.Tk()
root.title("Simulación de Ángulos Solares")

# Crear un marco para la gráfica polar
frame_polar = tk.Frame(root)
frame_polar.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

# Crear la figura y el eje polar
fig_polar = plt.Figure(figsize=(13, 7), dpi=100)
ax_polar = fig_polar.add_subplot(111, projection='polar')

# Configurar la dirección del ángulo (en sentido horario y con 0° en la parte superior)
ax_polar.set_theta_direction(-1)
ax_polar.set_theta_offset(np.pi / 2.0)

# Configurar los ejes
ax_polar.set_title('Trayectoria del sol en Coordenadas Polares')

# Función de inicialización para la animación polar
def init_polar():
    ax_polar.set_ylim(min(elevations), max(elevations))
    return []

# Función de actualización para la animación polar
def update_polar(frame):
    ax_polar.clear()  # Limpiar el gráfico en cada frame
    ax_polar.set_theta_direction(-1)
    ax_polar.set_theta_offset(np.pi / 2.0)
    ax_polar.set_ylim(min(elevations), max(elevations))
    ax_polar.set_title('Trayectoria del sol en Coordenadas Polares')
    
    theta = azimuths_rad[frame]  # Ángulo actual en radianes
    r = elevations[frame]  # Elevación actual
    
    # Superponer la imagen del sol en la ubicación actual
    ax_polar.imshow(sun_img, extent=(theta - 0.1, theta + 0.1, r - 0.1, r + 0.1), zorder=5)
    
    # Para mantener la trayectoria dibujada, cambiar color a rojo ('r') y el marcador a un punto verde ('go-')
    ax_polar.plot(azimuths_rad[:frame+1], elevations[:frame+1], color='red', marker='o', markerfacecolor='yellow', linestyle='-', markersize=5, label='Trayectoria del Sol')

    return []

# Crear el lienzo para la figura polar
canvas_polar = FigureCanvasTkAgg(fig_polar, master=frame_polar)
canvas_polar.draw()
canvas_polar.get_tk_widget().pack(fill=tk.BOTH, expand=True)

interval_time_ms = 500
ani_polar = animation.FuncAnimation(fig_polar, update_polar, frames=len(times), init_func=init_polar, interval=interval_time_ms, blit=True)

# Iniciar la interfaz gráfica de Tkinter
root.mainloop()

## Panel 3D

In [16]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.animation as animation
import tkinter as tk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# Función para crear los vértices del panel solar
def create_panel(center, width, height):
    w = width / 2
    h = height / 2
    vertices = np.array([
        [center[0] - w, center[1] - h, center[2]],  # Inferior izquierdo
        [center[0] + w, center[1] - h, center[2]],  # Inferior derecho
        [center[0] + w, center[1] + h, center[2]],  # Superior derecho
        [center[0] - w, center[1] + h, center[2]]   # Superior izquierdo
    ])
    return vertices

# Función para aplicar la rotación pitch (inclinación hacia adelante/atrás)
def apply_pitch(vertices, angle):
    angle = np.radians(angle)
    rotation_matrix = np.array([
        [1, 0, 0],
        [0, np.cos(angle), -np.sin(angle)],
        [0, np.sin(angle), np.cos(angle)]
    ])
    return vertices @ rotation_matrix.T

# Función para aplicar la rotación roll (inclinación lateral)
def apply_roll(vertices, angle):
    angle = np.radians(angle)
    rotation_matrix = np.array([
        [np.cos(angle), 0, np.sin(angle)],
        [0, 1, 0],
        [-np.sin(angle), 0, np.cos(angle)]
    ])
    return vertices @ rotation_matrix.T

# Función para inicializar la animación
def init():
    panel.set_verts([vertices])
    time_text.set_text('')  # Iniciar con texto vacío
    return panel, time_text

# Función para actualizar la animación con los valores de `beta` y `phi`
def update(frame):
    if frame < len(times):
        # Usar los valores calculados de pitch (beta) y roll (phi)
        rotated_vertices = apply_pitch(vertices, beta[frame])  # Rotación en pitch
        rotated_vertices = apply_roll(rotated_vertices, phi[frame])  # Rotación en roll

        # Actualizar la posición del panel
        panel.set_verts([rotated_vertices])

        # Actualizar la etiqueta de tiempo con el tiempo calculado
        current_time = times[frame].strftime("Tiempo: %H:%M")
        time_text.set_text(current_time)
    
    return panel, time_text

# Configurar la ventana de Tkinter
root = tk.Tk()
root.title("Simulación del Movimiento del Panel Solar con Tiempo")

# Crear un marco para la gráfica 3D
frame_3d = tk.Frame(root)
frame_3d.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

# Crear la figura y el eje 3D dentro de la ventana Tkinter
fig = plt.Figure(figsize=(10, 7), dpi=100)
ax = fig.add_subplot(111, projection='3d')

# Definir los parámetros del panel
center = [0, 0, 0]
width = 4
height = 2

# Crear los vértices del panel
vertices = create_panel(center, width, height)

# Inicializar la gráfica del panel
panel = Poly3DCollection([vertices], color='blue', linewidths=1, edgecolors='black')
ax.add_collection3d(panel)

# Configurar los límites de la gráfica
ax.set_xlim(-5, 5)
ax.set_ylim(-5, 5)
ax.set_zlim(-5, 5)

# Marcar la dirección del Norte (eje Y positivo)
ax.quiver(0, 0, 0, 0, 5, 0, color='red', linewidth=2)  # Flecha apuntando al norte
ax.text(0, 5, 0, "Norte", color='red', fontsize=12)    # Etiqueta de "Norte"

# Añadir texto para mostrar el tiempo
time_text = ax.text2D(0.05, 0.95, "", transform=ax.transAxes, color='black', fontsize=12)

# Crear el lienzo para la figura 3D
canvas3 = FigureCanvasTkAgg(fig, master=frame_3d)
canvas3.draw()
canvas3.get_tk_widget().pack(fill=tk.BOTH, expand=True)

# Crear la animación dentro de la ventana Tkinter, sin blit
interval_time_ms = 1000  # Un frame por segundo para sincronizar con el tiempo real
ani = animation.FuncAnimation(fig, update, frames=np.arange(0, len(times)), init_func=init, interval=interval_time_ms, blit=False)

# Iniciar la interfaz gráfica de Tkinter
root.mainloop()

In [17]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.animation as animation
import tkinter as tk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from datetime import datetime, timedelta
from pytz import timezone

# Supongamos que ya tienes la función calculate_solar_positions y los valores de times, beta, y phi

# Función para crear los vértices del panel solar
def create_panel(center, width, height):
    w = width / 2
    h = height / 2
    vertices = np.array([
        [center[0] - w, center[1], center[2] - h],  # Inferior izquierdo (Norte)
        [center[0] + w, center[1], center[2] - h],  # Inferior derecho (Norte)
        [center[0] + w, center[1], center[2] + h],  # Superior derecho (Sur)
        [center[0] - w, center[1], center[2] + h]   # Superior izquierdo (Sur)
    ])
    return vertices

# Función para aplicar la rotación roll (inclinación lateral) sobre el eje local X del panel
def apply_roll(vertices, angle):
    angle = np.radians(angle)
    rotation_matrix = np.array([
        [1, 0, 0],
        [0, np.cos(angle), -np.sin(angle)],
        [0, np.sin(angle), np.cos(angle)]
    ])
    return vertices @ rotation_matrix.T

# Función para aplicar la rotación pitch (inclinación hacia adelante/atrás) sobre el eje local Y del panel
def apply_pitch(vertices, angle):
    angle = np.radians(angle)
    rotation_matrix = np.array([
        [np.cos(angle), 0, np.sin(angle)],
        [0, 1, 0],
        [-np.sin(angle), 0, np.cos(angle)]
    ])
    return vertices @ rotation_matrix.T

# Función para inicializar la animación
def init():
    panel.set_verts([vertices])
    time_text.set_text('')  # Iniciar con texto vacío
    for scatter in scatters:
        scatter._offsets3d = (vertices[:, 0], vertices[:, 1], vertices[:, 2])
    return [panel] + scatters + [time_text]

# Función para actualizar la animación con los valores de `beta` y `phi`
def update(frame):
    if frame < len(times):
        # Aplicar roll y pitch
        rotated_vertices = apply_roll(vertices, phi[frame])
        rotated_vertices = apply_pitch(rotated_vertices, beta[frame])

        # Actualizar las posiciones del panel
        panel.set_verts([rotated_vertices])

        # Actualizar las posiciones de los puntos en los vértices
        for i, scatter in enumerate(scatters):
            scatter._offsets3d = (np.array([rotated_vertices[i, 0]]),
                                  np.array([rotated_vertices[i, 1]]),
                                  np.array([rotated_vertices[i, 2]]))

        # Actualizar la etiqueta de tiempo con el tiempo calculado
        current_time = times[frame].strftime("Tiempo: %H:%M")
        time_text.set_text(current_time)
    
    return [panel] + scatters + [time_text]

# Configurar la ventana de Tkinter
root = tk.Tk()
root.title("Simulación del Movimiento del Panel Solar")

# Crear un marco para la gráfica 3D
frame_3d = tk.Frame(root)
frame_3d.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

# Crear la figura y el eje 3D dentro de la ventana Tkinter
fig = plt.Figure(figsize=(10, 7), dpi=100)
ax = fig.add_subplot(111, projection='3d')

# Definir los parámetros del panel
center = [0, 0, 0]
width = 4
height = 2

# Crear los vértices del panel
vertices = create_panel(center, width, height)

# Inicializar el panel con colores diferentes para cada cara y un borde más grueso
panel = Poly3DCollection([vertices], facecolors=['blue', 'green'], linewidths=3, edgecolors='black')
ax.add_collection3d(panel)

# Agregar marcadores en los vértices para ayudar a guiar la visualización
scatters = [ax.scatter(vertice[0], vertice[1], vertice[2], color='red', s=50) for vertice in vertices]

# Configurar los límites de la gráfica
ax.set_xlim(-5, 5)
ax.set_ylim(-5, 5)
ax.set_zlim(-5, 5)

# Marcar la dirección del Norte (eje Y positivo)
ax.quiver(0, 0, 0, 0, 5, 0, color='red', linewidth=2)  # Flecha apuntando al norte
ax.text(0, 5, 0, "Norte", color='red', fontsize=12)    # Etiqueta de "Norte"

# Añadir texto para mostrar el tiempo
time_text = ax.text2D(0.05, 0.95, "", transform=ax.transAxes, color='black', fontsize=12)

# Crear el lienzo para la figura 3D
canvas3 = FigureCanvasTkAgg(fig, master=frame_3d)
canvas3.draw()
canvas3.get_tk_widget().pack(fill=tk.BOTH, expand=True)

# Crear la animación dentro de la ventana Tkinter, sin blit
interval_time_ms = 1000  # Un frame por segundo para sincronizar con el tiempo real
ani = animation.FuncAnimation(fig, update, frames=np.arange(0, len(times)), init_func=init, interval=interval_time_ms, blit=False)

# Iniciar la interfaz gráfica de Tkinter
root.mainloop()

In [24]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.animation as animation
import tkinter as tk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from datetime import datetime, timedelta
from pytz import timezone

# Supongamos que ya tienes la función calculate_solar_positions y los valores de times, beta, y phi

# Función para crear los vértices del panel solar
def create_panel(center, width, height):
    w = width / 2
    h = height / 2
    vertices = np.array([
        [center[0] - w, center[1] - h, center[2]],  # Inferior izquierdo
        [center[0] + w, center[1] - h, center[2]],  # Inferior derecho
        [center[0] + w, center[1] + h, center[2]],  # Superior derecho
        [center[0] - w, center[1] + h, center[2]]   # Superior izquierdo
    ])
    return vertices

# Función para aplicar ambas rotaciones (pitch y roll) usando matrices combinadas
def apply_rotations(vertices, pitch_angle, roll_angle):
    pitch_angle = np.radians(pitch_angle)
    roll_angle = np.radians(roll_angle)

    # Matriz de rotación para pitch (alrededor del eje Y)
    rotation_pitch = np.array([
        [np.cos(pitch_angle), 0, np.sin(pitch_angle)],
        [0, 1, 0],
        [-np.sin(pitch_angle), 0, np.cos(pitch_angle)]
    ])

    # Matriz de rotación para roll (alrededor del eje X)
    rotation_roll = np.array([
        [1, 0, 0],
        [0, np.cos(roll_angle), -np.sin(roll_angle)],
        [0, np.sin(roll_angle), np.cos(roll_angle)]
    ])

    # Combinar las matrices de rotación
    combined_rotation = rotation_pitch @ rotation_roll

    # Aplicar la rotación combinada a los vértices
    return vertices @ combined_rotation.T

# Función para inicializar la animación
def init():
    panel.set_verts([vertices])
    time_text.set_text('')  # Iniciar con texto vacío
    for scatter in scatters:
        scatter._offsets3d = (vertices[:, 0], vertices[:, 1], vertices[:, 2])
    return [panel] + scatters + [time_text]

# Función para actualizar la animación con los valores de `beta` y `phi`
def update(frame):
    if frame < len(times):
        # Aplicar rotaciones combinadas
        rotated_vertices = apply_rotations(vertices, beta[frame], phi[frame])

        # Actualizar las posiciones del panel
        panel.set_verts([rotated_vertices])

        # Actualizar las posiciones de los puntos en los vértices
        for i, scatter in enumerate(scatters):
            scatter._offsets3d = (np.array([rotated_vertices[i, 0]]),
                                  np.array([rotated_vertices[i, 1]]),
                                  np.array([rotated_vertices[i, 2]]))

        # Actualizar la etiqueta de tiempo con el tiempo calculado
        current_time = times[frame].strftime("Tiempo: %H:%M")
        time_text.set_text(current_time)
    
    return [panel] + scatters + [time_text]

# Configurar la ventana de Tkinter
root = tk.Tk()
root.title("Simulación del Movimiento del Panel Solar")

# Crear un marco para la gráfica 3D
frame_3d = tk.Frame(root)
frame_3d.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

# Crear la figura y el eje 3D dentro de la ventana Tkinter
fig = plt.Figure(figsize=(10, 7), dpi=100)
ax = fig.add_subplot(111, projection='3d')

# Definir los parámetros del panel
center = [0, 0, 0]
width = 4
height = 2

# Crear los vértices del panel
vertices = create_panel(center, width, height)

# Inicializar el panel con colores diferentes para cada cara y un borde más grueso
panel = Poly3DCollection([vertices], facecolors=['blue', 'green'], linewidths=3, edgecolors='black')
ax.add_collection3d(panel)

# Agregar marcadores en los vértices para ayudar a guiar la visualización
scatters = [ax.scatter(vertice[0], vertice[1], vertice[2], color='red', s=50) for vertice in vertices]

# Configurar los límites de la gráfica
ax.set_xlim(-5, 5)
ax.set_ylim(-5, 5)
ax.set_zlim(-5, 5)

# Marcar la dirección del Norte (eje Y positivo)
ax.quiver(0, 0, 0, 0, 5, 0, color='red', linewidth=2)  # Flecha apuntando al norte
ax.text(0, 5, 0, "Norte", color='red', fontsize=12)    # Etiqueta de "Norte"

# Añadir texto para mostrar el tiempo
time_text = ax.text2D(0.05, 0.95, "", transform=ax.transAxes, color='black', fontsize=12)

# Crear el lienzo para la figura 3D
canvas3 = FigureCanvasTkAgg(fig, master=frame_3d)
canvas3.draw()
canvas3.get_tk_widget().pack(fill=tk.BOTH, expand=True)

# Crear la animación dentro de la ventana Tkinter, sin blit
interval_time_ms = 1000  # Un frame por segundo para sincronizar con el tiempo real
ani = animation.FuncAnimation(fig, update, frames=np.arange(0, len(times)), init_func=init, interval=interval_time_ms, blit=False)

# Iniciar la interfaz gráfica de Tkinter
root.mainloop()